<a href="https://colab.research.google.com/github/p31x070/GPT/blob/main/PDF_Query_LLM-Adi%207273_MC_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=TLf90ipMzfE

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [4]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "..."

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
# location of the pdf file/files. 
reader = PdfReader('/content/drive/MyDrive/Zotero/Informativos STF/MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE 7.273 DISTRITO FEDERAL/Mendes -  PARTIDO SOCIALISTA BRASILEIRO - PSB NACIONAL  R.pdf')

In [45]:
reader

In [62]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

# raw_text

In [63]:
raw_text[:100]

'MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE  \n7.273 DISTRITO FEDERAL\nRELATOR :MIN. GILMA'

In [64]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [65]:
len(texts)

71

In [66]:
texts[0]

'MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE  \n7.273 DISTRITO FEDERAL\nRELATOR :MIN. GILMAR MENDES\nREQTE.(S):PARTIDO SOCIALISTA BRASILEIRO -  PSB \nNACIONAL \nADV.(A/S):RAFAEL ECHEVERRIA LOPES \nADV.(A/S):RAFAEL DE ALENCAR ARARIPE CARNEIRO \nINTDO.(A/S):CONGRESSO NACIONAL \nPROC.(A/S)(ES):ADVOGADO-GERAL DA UNIÃO \nINTDO.(A/S):PRESIDENTE DA REPÚBLICA \nPROC.(A/S)(ES):ADVOGADO-GERAL DA UNIÃO \nDECISÃO: Trata-se  de  ações  diretas  de  inconstitucionalidade  \npropostas contra o art. 39 da Lei 12.844/2013, em especial seu § 4º, que  \npermite a presunção de legalidade do ouro adquirido e a boa-fé da pessoa  \njurídica adquirente, nos seguintes termos:\n“ Art. 39. A prova da regularidade da primeira aquisição de  \nouro produzido sob qualquer regime de aproveitamento será  \nfeita com base em:\nI  –  nota  fiscal  emitida  por  cooperativa  ou,  no  caso  de  \npessoa física, recibo de venda e declaração de origem do ouro  \nemitido pelo vendedor identificando a área de lavr

In [67]:
texts[1]

'I  –  nota  fiscal  emitida  por  cooperativa  ou,  no  caso  de  \npessoa física, recibo de venda e declaração de origem do ouro  \nemitido pelo vendedor identificando a área de lavra, o Estado  \nou Distrito Federal e o Município de origem do ouro, o número  \ndo  processo  administrativo  no  órgão  gestor  de  recursos  \nminerais e o número do título autorizativo de extração; e\nII  –  nota  fiscal  de  aquisição  emitida  pela  instituição  \nautorizada pelo Banco Central do Brasil a realizar a compra do  \nouro.\n§ 1º Para os efeitos deste artigo, a instituição legalmente  \nautorizada  a  realizar  a compra  de  ouro  deverá  cadastrar  os  \ndados de identificação do vendedor, tais como nome, número  \nde  inscrição  no  Cadastro  de  Pessoa  Física  do  Ministério  da  \nFazenda – CPF  ou Cadastro Nacional da Pessoa Jurídica do  \nMinistério da Fazenda – CNPJ, e o número de registro no órgão  \nde registro do comércio da sede do vendedor.'

In [68]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [69]:
docsearch = FAISS.from_texts(texts, embeddings)

In [70]:
docsearch

In [71]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [72]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

# Resumo do caso

In [73]:
query = "Faça um resumo do caso"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O Partido Socialista Brasileiro, a Rede Sustentabilidade e o Partido Verde apresentaram uma ação direta de inconstitucionalidade contra o art. 39 da Lei 12.844/2013. O artigo em questão permite a presunção de legalidade do ouro adquirido e a boa-fé da pessoa jurídica adquirente. O juiz decidiu que os requisitos para a concessão da medida cautelar estavam presentes, justificando o deferimento da medida cautelar. Ele também considerou que as alterações promovidas pela Lei de 2013 permitiram a expansão do comércio ilegal, fortalecendo as atividades de garimpo ilegal, desmatamento, contaminação de rios e violência nas regiões de garimpo, atingindo os povos indígenas das áreas afetadas.'

# Norma impugnada

In [74]:
query = "qual a norma impugnada. transcreva o texto exato do dispositivo"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O dispositivo impugnado é o art. 39 da Lei 12.844/2013.'

# Presumem-se a legalidade...

In [79]:
query = "transcreva o §4 do art. 39 da Lei 12.844/2013."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' § 4º Presumem-se a legalidade do ouro adquirido e a boa-fé da pessoa jurídica adquirente quando as informações mencionadas neste artigo, prestadas pelo vendedor, estiverem devidamente arquivadas na instituição legalmente autorizada a realizar a compra de ouro.'

# Sumário

In [80]:
query = "Escreva uma notícia de Jornal baseada neste voto do Ministro Gilmar Mendes"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O Ministro Gilmar Mendes votou em favor da adoção de um marco normativo para a fiscalização do comércio de ouro, visando a verificação da origem legal daquele adquirido por DTVMs. Além disso, o Ministro votou a favor de medidas legislativas, regulatórias e administrativas que inviabilizem a aquisição de ouro extraído de áreas de proteção ambiental e de Terras Indígenas. O Ministro também votou a favor de que a medida cautelar seja julgada no Plenário Virtual, em cumprimento ao disposto no artigo 21, V do RI/STF.'

# O artigo 21, V do RI/STF prevê o referendo

In [82]:
query = "o que prevê o art 21, V do RI/STF"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O artigo 21, V do RI/STF prevê o referendo desta medida cautelar para julgamento no Plenário Virtual.'

In [84]:
query = "como citar esse esta decisão?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Esta decisão pode ser citada como Adi 7273 MC / DF.'

# palavras chave

In [87]:
query = "crie palavras chave para o artigo. Apresente o resultado em tópicos"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' \n- Lei 9.868/1999 \n- Câmara dos Deputados \n- Presidente da República \n- Autoridade Geral da União \n- Subchefia para Assuntos Jurídicos \n- Business Intelligence \n- Sistema Financeiro \n- Ouro Ilegal\n- Tecnologia de Análise de Imagens de Satélite \n- Mandamentos Constitucionais\n- Medida Cautelar'

 
- Lei 9.868/1999 
- Câmara dos Deputados 
- Presidente da República 
- Autoridade Geral da União 
- Subchefia para Assuntos Jurídicos 
- Business Intelligence 
- Sistema Financeiro 
- Ouro Ilegal
- Tecnologia de Análise de Imagens de Satélite 
- Mandamentos Constitucionais
- Medida Cautelar

# Citação - Ouro ilegal

In [89]:
query = "extraia uma citação do texto que esteja relacionada as palavras chave ouro ilegal. Indique a página"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' "Sobre a fragilidade provocada na cadeia de controle e rastreabilidade da produção e comércio de ouro oriundo de regiões auríferas gerada com a presunção legal de boa-fé do § 4º do artigo 39 da Lei nº 12.844/2013, o COAF, em discussões da ENCCLA, Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro, a principal rede de articulação institucional brasileira para o arranjo, discussões, formulação e concretização de políticas públicas e soluções de enfrentamento à corrupção e à lavagem de dinheiro, posicionou-se no sentido (página 16).'

# fragilidade do arcabouço normativo

In [91]:
query = "o que o texto trata sobre: Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O texto trata sobre a fragilidade do arcabouço normativo da cadeia produtiva do ouro que estabeleceu uma condição desafiadora para os controles de prevenção à lavagem de dinheiro do Estado. Além disso, menciona que o COAF, a Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro, posicionou-se no sentido de que a possibilidade de controle da origem do metal é muito limitada.'

In [92]:
query = "relacione os fatos elencados como argumentos para a decisão"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' A decisão foi baseada na verossimilhança do direito, na relevância da matéria em debate, na discricionariedade legislativa, na inépcia da inicial, na opção legislativa, no espaço de discricionariedade legislativa e nos danos causados pelo garimpo ilegal.'

# Verossimilhança do direito

In [93]:
query = "o que fundamenta a verossimilhança do direito nesta decisão?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O fundamento da verossimilhança do direito nesta decisão é o princípio da proporcionalidade e a jurisprudência do Supremo Tribunal Federal, que declara a inconstitucionalidade de normas que afastam ou enfraquecem o controle prévio de empreendimentos que impactam o meio ambiente.'

In [95]:
query = "quais normas que afastam ou enfraquecem o controle prévio de empreendimentos que impactam o meio ambiente."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Normas que dispensam a exigência de licenciamento para atividades potencialmente poluidoras, como o artigo 10 da Lei 2.713/2013 do Estado do Tocantins.'

 O fundamento da verossimilhança do direito nesta decisão é o princípio da proporcionalidade e a jurisprudência do Supremo Tribunal Federal, que declara a inconstitucionalidade de normas que afastam ou enfraquecem o controle prévio de empreendimentos que impactam o meio ambiente, de que são exemplo as  Normas que dispensam a exigência de licenciamento para atividades potencialmente poluidoras, como o artigo 10 da Lei 2.713/2013 do Estado do Tocantins.
 